In [7]:
import pandas as pd

## Testin Zone:
Testing classes used in the agent

In [9]:
def determine_directions(point1, point2, size=21):
        """ Given two points determine the closest directions to take to get to the point. """
        x1, y1 = point1.x, point1.y
        x2, y2 = point2.x, point2.y

         # For both x and y they are two type of paths to take
        diff_x_1 = abs(x2 - x1) 
        diff_x_2 = abs(size - x2 + x1)
        diff_y_1 = abs(y2 - y1)
        diff_y_2 = abs(size - y2 + y1)
        # Given that x1=x2 or y1=y2 then return None, so we know 
        # no movement was needed in that axis.
        best_x, best_y = None, None

        if diff_x_1 > diff_x_2:
            if x2 - x1 > 0: 
                best_x = "E"
            else:
                best_x = "W"
        else:
            if x2 - x1 > 0: 
                best_x = "W"
            else:
                best_x = "E"

        if diff_y_1 > diff_y_2:
            if y2 - y1 > 0:
                best_y = "N"
            else:
                best_y = "S"
        else:
            if y2 - y1 > 0:
                best_y = "S"
            else:
                best_y = "N"
                
        if best_x is None: best_x = random.choice(['E', 'W'])
        if best_y is None: best_y = random.choice(['S', 'N'])
        
        return best_x, best_y


def grid(cell):
    """ Returns a dictionary of cells which are in 4 moves distance of the given cell """
    # The directions that are one move away
    north, south, west, east = cell.north, cell.south, cell.west, cell.east
    # The directions that are two moves away
    nn, ss, ww, ee = north.north, south.south, west.west, east.east

    return {
        'N': north, 'S': south, 'W': west, 'E': east, 

        'NW': north.west, 'NE': north.east, 'SW': south.west, 'SE': south.east, 'WW': ww, 'EE': ee, 'NN': nn, 'SS': ss ,
        
        'NEN': nn.east, 'NWN': nn.west, 'SES': ss.east, 'SWS': ss.west, 
        'SEE': ee.south, 'NEE': ee.north, 'SWW': ww.south, 'NWW': ww.north, 
        'SSS': ss.south, 'EEE': ee.east, 'WWW': ww.west, 'NNN': nn.north, 
        
        'NNNN': nn.north.north, 'SSSS': ss.south.south, 'WWWW': ww.west.west, 'EEEE': ee.east.east,
        'SEES': ee.south.south , 'NEEN': ee.north.north, 'NWWN': ww.north.north, 'SWWS': ww.south.south,
        'WWWS': ww.west.south, 'EEES': ee.east.south, 'EEEN': ee.east.north, 'WWWN': ww.west.north,
        'SWSS': ss.south.west, 'SESS': ss.south.east, 'NENN': nn.north.east, 'NWNN': nn.north.west 
    }        


def count_moves(point1, point2, size=21):
    """ 
        Returns the minimum number of between moves 
        to go from point1 to point2.
    """
    # Break the points into coordinates
    x1, y1 = point1.x, point1.y
    x2, y2 = point2.x, point2.y
    
    # For both x and y they are two type of paths to take
    diff_x_1 = abs(x2 - x1) 
    diff_x_2 = abs(size + x2 - x1)
    diff_y_1 = abs(y2 - y1)
    diff_y_2 = abs(size + y2 - y1)
    
    opt1 = diff_x_1 + diff_y_1
    opt2 = diff_x_1 + diff_y_2
    opt3 = diff_x_2 + diff_y_1
    opt4 = diff_x_2 + diff_y_2
    
    return min(opt1, opt2, opt3, opt4)


def log(text, step=1):
    if step == 0:
        with open("log.txt", "w") as text_file:
            text = str(text) + '\n'
            text_file.write(text)
    else:
        with open("log.txt", "a") as text_file:
            text = str(text) + '\n'
            text_file.write(text)
def determine_directions(point1, point2, size=21):
        """ Given two points determine the closest directions to take to get to the point. """
        x1, y1 = point1.x, point1.y
        x2, y2 = point2.x, point2.y

         # For both x and y they are two type of paths to take
        diff_x_1 = abs(x2 - x1) 
        diff_x_2 = abs(size - x2 + x1)
        diff_y_1 = abs(y2 - y1)
        diff_y_2 = abs(size - y2 + y1)
        # Given that x1=x2 or y1=y2 then return None, so we know 
        # no movement was needed in that axis.
        best_x, best_y = None, None

        if diff_x_1 > diff_x_2:
            if x2 - x1 > 0: 
                best_x = "E"
            else:
                best_x = "W"
        else:
            if x2 - x1 > 0: 
                best_x = "W"
            else:
                best_x = "E"

        if diff_y_1 > diff_y_2:
            if y2 - y1 > 0:
                best_y = "N"
            else:
                best_y = "S"
        else:
            if y2 - y1 > 0:
                best_y = "S"
            else:
                best_y = "N"
                
        if best_x is None: best_x = random.choice(['E', 'W'])
        if best_y is None: best_y = random.choice(['S', 'N'])
        
        return best_x, best_y


def grid(cell):
    """ Returns a dictionary of cells which are in 4 moves distance of the given cell """
    # The directions that are one move away
    north, south, west, east = cell.north, cell.south, cell.west, cell.east
    # The directions that are two moves away
    nn, ss, ww, ee = north.north, south.south, west.west, east.east

    return {
        'N': north, 'S': south, 'W': west, 'E': east, 

        'NW': north.west, 'NE': north.east, 'SW': south.west, 'SE': south.east, 'WW': ww, 'EE': ee, 'NN': nn, 'SS': ss ,
        
        'NEN': nn.east, 'NWN': nn.west, 'SES': ss.east, 'SWS': ss.west, 
        'SEE': ee.south, 'NEE': ee.north, 'SWW': ww.south, 'NWW': ww.north, 
        'SSS': ss.south, 'EEE': ee.east, 'WWW': ww.west, 'NNN': nn.north, 
        
        'NNNN': nn.north.north, 'SSSS': ss.south.south, 'WWWW': ww.west.west, 'EEEE': ee.east.east,
        'SEES': ee.south.south , 'NEEN': ee.north.north, 'NWWN': ww.north.north, 'SWWS': ww.south.south,
        'WWWS': ww.west.south, 'EEES': ee.east.south, 'EEEN': ee.east.north, 'WWWN': ww.west.north,
        'SWSS': ss.south.west, 'SESS': ss.south.east, 'NENN': nn.north.east, 'NWNN': nn.north.west 
    }        


def count_moves(point1, point2, size=21):
    """ 
        Returns the minimum number of between moves 
        to go from point1 to point2.
    """
    # Break the points into coordinates
    x1, y1 = point1.x, point1.y
    x2, y2 = point2.x, point2.y
    
    # For both x and y they are two type of paths to take
    diff_x_1 = abs(x2 - x1) 
    diff_x_2 = abs(size + x2 - x1)
    diff_y_1 = abs(y2 - y1)
    diff_y_2 = abs(size + y2 - y1)
    
    opt1 = diff_x_1 + diff_y_1
    opt2 = diff_x_1 + diff_y_2
    opt3 = diff_x_2 + diff_y_1
    opt4 = diff_x_2 + diff_y_2
    
    return min(opt1, opt2, opt3, opt4)


def log(text, step=1):
    if step == 0:
        with open("log.txt", "w") as text_file:
            text = str(text) + '\n'
            text_file.write(text)
    else:
        with open("log.txt", "a") as text_file:
            text = str(text) + '\n'
            text_file.write(text)


In [10]:
class Locator:
    """ This object returns dataframes containing the information about other ships/shipyards on the map and also in the given ship's grid. """
    def __init__(self, board, ship):
        self.board = board
        self.ship = ship
        self.ship_position = ship.position
        # Get the grid
        self.grid = grid(ship.cell)
        
    def get_ship_info(self):
        """ Returns the info about ships in all of the board. """
        ships_info = {}
        
        for ship_id, ship in self.board.ships.items():
            base_info = {
                "my_ship": 0, "cargo": 0, 
                "moves": 0, "position": (ship.position.x, ship.position.y)
            }
            
            base_info['cargo'] = ship.cell.halite
            base_info['moves'] = count_moves(self.ship.position, ship.position)
            base_info['dirX'], base_info['dirY'] = determine_directions(self.ship_position, ship.position)
            if ship_id in self.board.current_player.ship_ids and ship.id != self.ship.id:
                base_info['my_ship'] = 1
                ships_info[ship_id] = base_info
            elif not(ship_id in self.board.current_player.ship_ids):
                ships_info[ship_id] = base_info
        
        
        return  pd.DataFrame(ships_info)
    
    
    def get_shipyard_info(self):
        """ Returns the info about shipyards in all of the board. """
        shipyards_info = {}
        
        for shipyard_id, shipyard in self.board.shipyards.items():
            base_info = {
                "my_shipyard": 0, "player_halite": 0, 
                "moves": 0, "position": (shipyard.position.x, shipyard.position.y),
            }
            
            base_info['dirX'], base_info['dirY'] = determine_directions(self.ship_position, shipyard.position)
            base_info['player_halite'] = shipyard.player.halite
            base_info['moves'] = count_moves(self.ship.position, shipyard.position)
            
            if shipyard_id in self.board.current_player.shipyard_ids:
                base_info['my_shipyard'] = 1
                
                shipyards_info[shipyard_id] = base_info
            else:
                shipyards_info[shipyard_id] = base_info
        
        return pd.DataFrame(shipyards_info)
        
        
    def generate_grid_df(self):
        """ Generates a Dataframe describing the information of objects and cells in the 5x5 grid of the ship. """
        all_dirs = {}
        
        for direction, cell in self.grid.items():
            
            base_info = {
                "ship_id": None, "shipyard_id": None, 
                "my_ship": 0, "my_shipyard": 0,
                "halite": 0, "moves": 0
            }
            
            if cell.ship != None: 
                base_info["ship_id"] = cell.ship.id
                if cell.ship.id in self.ship.player.ship_ids:
                    base_info["my_ship"] = 1
    
            if cell.shipyard != None: 
                base_info["shipyard_id"] = cell.shipyard.id
                if cell.shipyard.id in self.ship.player.shipyard_ids:
                    base_info['my_shipyard'] = 1
                    
            base_info['halite'] = cell.halite
            # The number of letters in the direction would indicate the number of moves needed to get there
            base_info['moves'] = len(direction)
            
            all_dirs[direction] = base_info
            
        return pd.DataFrame(all_dirs)
        

In [8]:
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *

# Create a test environment for use later
board_size = 21
environment = make("halite", configuration={"size": board_size, "startingHalite": 30000}, debug=True)
agent_count = 4
environment.reset(agent_count)
state = environment.state[0]

In [11]:
# Change the behviour of my own movement from board.current_player
board = Board(state.observation, environment.configuration)

board.current_player.ships[0].next_action = ShipAction.CONVERT
board.ships['0-2'].next_action = ShipAction.CONVERT
board.ships['0-3'].next_action = ShipAction.CONVERT
board.ships['0-4'].next_action = ShipAction.CONVERT
board = board.next()
board.current_player.shipyards[0].next_action = ShipyardAction.SPAWN
board.shipyards['1-2'].next_action = ShipyardAction.SPAWN
board.shipyards['1-3'].next_action = ShipyardAction.SPAWN
board.shipyards['1-4'].next_action = ShipyardAction.SPAWN
board = board.next()

# Current-Player
board.current_player.ships[0].next_action  = ShipAction.EAST
board.current_player.shipyards[0].next_action = ShipyardAction.SPAWN
# Opponents
board.ships['2-2'].next_action = ShipAction.NORTH
board.shipyards['1-2'].next_action = ShipyardAction.SPAWN
board.ships['2-3'].next_action = ShipAction.WEST
board.shipyards['1-3'].next_action = ShipyardAction.SPAWN
board.ships['2-4'].next_action = ShipAction.EAST
board.shipyards['1-4'].next_action = ShipyardAction.SPAWN
board = board.next()

# Current-Player
board.current_player.ships[0].next_action  = ShipAction.NORTH
board.current_player.ships[1].next_action  = ShipAction.SOUTH
board.current_player.shipyards[0].next_action = ShipyardAction.SPAWN
# Opponents
board.ships['2-2'].next_action = ShipAction.WEST
board.shipyards['1-2'].next_action = ShipyardAction.SPAWN
board.ships['2-3'].next_action = ShipAction.NORTH
board.shipyards['1-3'].next_action = ShipyardAction.SPAWN
board.ships['2-4'].next_action = ShipAction.SOUTH
board.shipyards['1-4'].next_action = ShipyardAction.SPAWN
board = board.next()

# Current-Player
board.current_player.ships[0].next_action  = ShipAction.EAST
board.current_player.ships[1].next_action  = ShipAction.SOUTH
board.current_player.ships[2].next_action  = ShipAction.EAST
# Opponents
board.ships['2-2'].next_action = ShipAction.WEST
board.ships['2-3'].next_action = ShipAction.EAST
board.ships['2-4'].next_action = ShipAction.SOUTH
board = board.next()

# Current-Player
board.current_player.ships[0].next_action  = ShipAction.EAST
board.current_player.ships[1].next_action  = ShipAction.SOUTH
board.current_player.ships[2].next_action  = ShipAction.EAST
# Opponents
board.ships['2-2'].next_action = ShipAction.WEST
board.ships['2-3'].next_action = ShipAction.EAST
board.ships['2-4'].next_action = ShipAction.SOUTH
board = board.next()

# Current-Player
board.current_player.ships[0].next_action  = ShipAction.EAST
board.current_player.ships[1].next_action  = ShipAction.SOUTH
board.current_player.ships[2].next_action  = ShipAction.EAST
# Opponents
board.ships['2-2'].next_action = ShipAction.WEST
board.ships['2-3'].next_action = ShipAction.EAST
board.ships['2-4'].next_action = ShipAction.SOUTH
board = board.next()

# Current-Player
board.current_player.ships[0].next_action  = ShipAction.EAST
board.current_player.ships[1].next_action  = ShipAction.SOUTH
board.current_player.ships[2].next_action  = ShipAction.EAST
# Opponents
board.ships['2-2'].next_action = ShipAction.NORTH
board.ships['2-3'].next_action = ShipAction.EAST
board.ships['2-4'].next_action = ShipAction.SOUTH
board = board.next()

board.current_player.ships[0].next_action = ShipAction.CONVERT
board = board.next()

ship = board.current_player.ships[0]
loc = Locator(board, ship)

In [12]:
print(board)

| 2 | 0 | 2 | 0 | 0 | 0 | 3 | 0 | 2 | 0 | 9 | 0 | 2 | 0 | 3 | 0 | 0 | 0 | 2 | 0 | 2 |
| 1 | 0 | 1 | 0 | 2 | 1 | 0 | 0 | 2 | 6 | 2 | 6 | 2 | 0 | 0 | 1 | 2 | 0 | 1 | 0 | 1 |
| 4 | 0 | 4 | 0 | 6 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 6 | 0 | 4 | 0 | 4 |
| 0 | 0 | 0 | 0 | 0 | 4 | 0 | 0 | 0 | 0 | 0 |b0 | 0 | 0 | 0 | 4 | 0 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 3 | 0 | 0 | 0 | 0A| 0 | 0 | 0 | 3 | 0 | 0 | 0 | 0 | 0 | 1 |
| 5 | 0 | 0 | 0 | 0 | 0A| 0 | 0 | 0 |a0 | 0 | 0 | 0 | 0 | 0 | 0B| 0 | 0 | 0 | 0 | 5 |
| 0 | 0 | 0 | 6 | 0 | 3 | 0 | 1 | 5 | 0 | 1 | 0 | 5 | 1 | 0 | 3 | 0 | 6 | 0 | 0 | 0 |
| 0 | 5 | 0 | 0 | 2 | 0 | 0 | 1 | 8 | 0 | 1 | 0 | 8 | 1 | 0 | 0 | 2 | 0 | 0 | 5 | 0 |
| 0 | 0 | 1 | 0 | 0 | 0 | 5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 5 | 0 | 0 | 0 | 1 | 0 | 0 |
| 0 | 4 | 0 | 0 | 1 | 0 | 0 | 2 | 0 | 0 | 6 | 0 | 0 | 2 | 0 | 0 | 1 | 0 | 0 | 4 | 0 |
| 0 | 8 | 0 | 0 | 1 |a5 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 2 | 0 | 7 | 1 | 0 | 0 | 8 | 0 |
| 0 | 4 | 0 | 0 | 1 | 0 | 0 | 2 | 0 | 0 | 6 | 0 | 0 | 

In [13]:
board.ships

{'3-1': <kaggle_environments.envs.halite.helpers.Ship at 0x7f3eaf9b26a0>,
 '4-1': <kaggle_environments.envs.halite.helpers.Ship at 0x7f3eaf9b2fd0>,
 '2-2': <kaggle_environments.envs.halite.helpers.Ship at 0x7f3eaf9b22b0>,
 '2-3': <kaggle_environments.envs.halite.helpers.Ship at 0x7f3eaf9b2640>,
 '2-4': <kaggle_environments.envs.halite.helpers.Ship at 0x7f3eaf9b2310>}

In [43]:
board.shipyards

{'1-1': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7f895e26a8b0>,
 '1-2': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7f895e26a9a0>,
 '1-3': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7f895e26aa90>,
 '1-4': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7f895e26ab80>,
 '9-1': <kaggle_environments.envs.halite.helpers.Shipyard at 0x7f89691b4160>}

In [44]:
board.current_player.ships

In [14]:
yard = board.shipyards['1-4']
l = Locator(board, yard).generate_grid_df().T

In [15]:
l.T

,N,S,W,E,NW,NE,SW,SE,WW,EE,...,NWWN,SWWS,WWWS,EEES,EEEN,WWWN,SWSS,SESS,NENN,NWNN
ship_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shipyard_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
my_ship,0.000,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000
my_shipyard,0.000,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000
halite,209.141,26.293,0.0,0.0,8.368,0.0,170.899,0.0,0.0,0.0,...,77.681,0.0,0.0,0.0,0.0,282.041,0.0,326.894,0.0,302.358
moves,1.000,1.000,1.0,1.0,2.000,2.0,2.000,2.0,2.0,2.0,...,4.000,4.0,4.0,4.0,4.0,4.000,4.0,4.000,4.0,4.000


In [16]:
z = loc.get_shipyard_info()

In [17]:
z

,1-1,1-2,1-3,1-4,9-1
my_shipyard,1,0,0,0,1
player_halite,2500,3000,3000,3000,2500
moves,5,15,5,15,11
position,"(5, 15)","(15, 15)","(5, 5)","(15, 5)","(10, 16)"
dirX,E,W,E,W,W
dirY,S,S,N,N,S


In [47]:
z.T[z.T['my_shipyard'] == 1]

,my_shipyard,player_halite,moves,position,dirX,dirY
1-1,1,2500,5,"(5, 15)",E,S
9-1,1,2500,11,"(10, 16)",W,S


In [26]:
min_val = z.T.loc[z.T['my_shipyard'] == 1]['moves'].min()
min_val

5

In [20]:
my_shipyards = z.T[z.T['my_shipyard'] == 1]

In [31]:
list(my_shipyards[my_shipyards['moves'] == min_val].T.columns)[0]

'1-1'

In [51]:
list(z.T.loc[z.T['moves'] == min_val].T.columns)[0]

'1-1'

In [114]:
z.T[z.T['my_shipyard'] == 1]

,my_shipyard,player_halite,moves,position,dirX,dirY
1-1,1,3000,3,"(5, 15)",E,N


In [11]:
loc.generate_grid_df()

,N,S,W,E,NW,NE,SW,SE,WW,EE,...,NWWN,SWWS,WWWS,EEES,EEEN,WWWN,SWSS,SESS,NENN,NWNN
ship_id,NaN,NaN,NaN,NaN,NaN,NaN,4-1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shipyard_id,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
my_ship,0.0,0.000,0.0,0.000,0.0,0.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.0,0.000,0.0
my_shipyard,0.0,0.000,0.0,0.000,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.0,0.000,0.0
halite,0.0,205.359,0.0,422.862,162.3,0.0,308.811,0.0,0.0,0.0,...,0.0,0.0,0.0,126.97,0.0,257.25,100.471,0.0,115.929,0.0
moves,1.0,1.000,1.0,1.000,2.0,2.0,2,2.0,2.0,2.0,...,4.0,4.0,4.0,4.00,4.0,4.00,4.000,4.0,4.000,4.0


In [15]:
d = loc.get_ship_info()
d

,3-1,4-1,2-2,2-3,2-4
my_ship,1,1,0,0,0
cargo,0,308.811,0,0,252.206
moves,5,2,6,12,17
position,"(5, 13)","(6, 15)","(13, 16)","(5, 6)","(16, 3)"


In [23]:
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *

acts = {
    "N": ShipAction.NORTH, 'S': ShipAction.SOUTH,
    'W': ShipAction.WEST , 'E' : ShipAction.EAST,
    'spawn': ShipyardAction.SPAWN, 'convert': ShipAction.CONVERT,
    'mine': None
}

# Create a test environment for use later
board_size = 21
environment = make("halite", configuration={"size": board_size, "startingHalite": 5000}, debug=True)
agent_count = 4
environment.reset(agent_count)
state = environment.state[0]
board = Board(state.observation, environment.configuration)

for i in range(12):
    player = board.current_player
    print(board.observation['step'], '#####################################')
    
    for ship in player.ships:
        log(str(ship.position) + ', id=' + ship.id)
        decider = Decision_Ship(board, ship)
        ship.next_action = decider.determine()
        print('----------------------')
    for shipyard in player.shipyards:
        if player.halite > 2000:
            shipyard.next_action = acts['spawn']
                
    print(player.next_actions)
    
    board = board.next()

0 #####################################
----------------------
{'0-1': 'NORTH'}
1 #####################################
----------------------
{'0-1': 'NORTH'}
2 #####################################
----------------------
{'0-1': 'NORTH'}
3 #####################################
----------------------
{'0-1': 'NORTH'}
4 #####################################
----------------------
{'0-1': 'NORTH'}
5 #####################################
----------------------
{'0-1': 'NORTH'}
6 #####################################
----------------------
{'0-1': 'NORTH'}
7 #####################################
----------------------
{'0-1': 'NORTH'}
8 #####################################
----------------------
{'0-1': 'NORTH'}
9 #####################################
----------------------
{'0-1': 'NORTH'}
10 #####################################
----------------------
{'0-1': 'NORTH'}
11 #####################################
----------------------
{'0-1': 'NORTH'}


# Actual writting zone:

In [111]:
%%writefile current.py


Overwriting current.py


In [5]:
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *

env = make("halite", debug=True)
env.run(["random", "current.py", 'random', 'attacker_c_2.py'])
env.render(mode="ipython", width=800, height=600)